In [24]:
!pip install opencv-python
import boto3
import cv2
import numpy as np
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

In [25]:
# S3 클라이언트 생성
s3 = boto3.client('s3')

# 버킷 및 폴더 정보
source_bucket = "smwu-cv-data"
source_folder = "data/다이캐스팅/g2/dataset_2023-10-17/NG/1"
output_bucket = "smwu-cv-team-2-s3"
output_folder = "preprocessed-data/Angle/"

In [26]:
def process_image(image_bytes):
    # 흑백 이미지 로드
    image = np.array(Image.open(BytesIO(image_bytes)))
    
    # 엣지 검출
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    
    # Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)
    angle = 0
    if lines is not None:
        for rho, theta in lines[0]:
            angle = (theta - np.pi / 2) * 180 / np.pi
            break

    # 이미지 회전
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    return rotated

In [27]:
def process_and_upload_images():
    # S3에서 목록 가져오기
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)
    if 'Contents' not in response:
        print("No files found in the source folder.")
        return

    for obj in response['Contents']:
        key = obj['Key']
        if key.endswith(('jpg', 'jpeg', 'png')):  # 이미지 파일만 처리
            print(f"Processing: {key}")
            
            # S3에서 이미지 다운로드
            file_obj = s3.get_object(Bucket=source_bucket, Key=key)
            image_bytes = file_obj['Body'].read()
            
            # 이미지 처리
            rotated_image = process_image(image_bytes)
            
            # OpenCV 이미지를 PIL 이미지로 변환
            pil_image = Image.fromarray(rotated_image)
            
            # 처리된 이미지를 S3로 업로드
            buffer = BytesIO()
            pil_image.save(buffer, format="JPEG")  # PIL 객체로 저장
            buffer.seek(0)
            
            # 새 키 생성 및 업로드
            output_key = output_folder + key.split('/')[-1]
            s3.put_object(Bucket=output_bucket, Key=output_key, Body=buffer, ContentType='image/jpeg')
            print(f"Uploaded: {output_key}")

In [28]:
process_and_upload_images()

Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-202440121.png
Uploaded: preprocessed-data/Angle/20231013-202440121.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-204432370.png
Uploaded: preprocessed-data/Angle/20231013-204432370.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-210521050.png
Uploaded: preprocessed-data/Angle/20231013-210521050.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-210600740.png
Uploaded: preprocessed-data/Angle/20231013-210600740.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-210621893.png
Uploaded: preprocessed-data/Angle/20231013-210621893.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-210643147.png
Uploaded: preprocessed-data/Angle/20231013-210643147.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-210704301.png
Uploaded: preprocessed-data/Angle/20231013-210704301.png
Processing: data/다이캐스팅/g2/dataset_2023-10-17/NG/1/20231013-210725454.png
Uploaded: preproc